In [51]:
# Start an edit session
edit = arcpy.da.Editor(workspace)
edit.startEditing(False, True)

# Stop all edit sessions
edit.stopEditing(True)

print("All editing sessions have been stopped.")

All editing sessions have been stopped.


In [52]:
import arcpy

# Set the workspace
workspace = r"D:/ECCC/MyProject/MyProject.gdb"
arcpy.env.workspace = workspace

# Define the name of your shapefile and fields
shapefile = "SoilParticles2"
clay_field = "clay"
silt_field = "silt"
sand_field = "sand"
texture_field = "texture"

# Check if the texture field exists, and if not, add it
if not arcpy.ListFields(shapefile, texture_field):
    arcpy.AddField_management(shapefile, texture_field, "TEXT")

# Start an edit session
edit = arcpy.da.Editor(workspace)
edit.startEditing(False, True)
edit.startOperation()

# Primary classification function based on USDA soil texture triangle
def primary_classify_texture(clay, silt, sand):
    if sand >= 85 and silt <= 10 and clay <= 15:
        return "Sand"
    elif sand >= 70 and sand < 90 and silt <= 30 and clay <= 15:
        return "Loamy Sand"
    elif sand >= 43 and sand <= 85 and silt >= 10 and silt <= 50 and clay <= 20:
        return "Sandy Loam"
    elif sand >= 23 and sand <= 52 and silt >= 28 and silt <= 50 and clay >= 7 and clay <= 27:
        return "Loam"
    elif silt >= 80 and sand <= 20 and clay <= 12:
        return "Silt"
    elif silt >= 50 and silt < 80 and clay <= 27 and sand <= 50:
        return "Silt Loam"
    elif sand >= 45 and sand <= 65 and clay >= 20 and clay < 35 and silt <= 28:
        return "Sandy Clay Loam"
    elif clay >= 27 and clay <= 40 and sand >= 20 and sand <= 45 and silt <= 40:
        return "Clay Loam"
    elif clay >= 27 and clay < 40 and silt >= 40 and silt <= 73 and sand <= 20:
        return "Silty Clay Loam"
    elif clay >= 35 and sand >= 45 and silt < 20:
        return "Sandy Clay"
    elif clay >= 40 and silt >= 40 and sand <= 20:
        return "Silty Clay"
    elif clay >= 40 and silt <= 40 and sand <= 45:
        return "Clay"
    else:
        return "Unknown"  # Initially mark as Unknown if not classified

# Secondary classification for "Unknown" values using additional conditions for intermediare class : less than 3% of the landscape
def secondary_classify_texture(clay, silt, sand):
    if 50 < sand < 70 and silt < 30 and clay < 30:
        return "Sandy Clay Loam"
    elif 40 < sand < 80 and silt < 20 and clay < 30:
        return "Sandy Loam"
    elif 30 < sand < 40 and 30 < silt < 40 and clay < 30:
        return "Loam"
    elif sand < 43 and silt > 50 and clay < 27:
        return "Silt Loam"
    elif clay > 35 and sand > 50 and silt < 20:
        return "Sandy Clay"
    elif 30 > sand  and 30 <= silt and 30 <= clay :
        return "Clay Loam"
    elif sand > 67 and silt < 10 and 20 <= clay <= 30:
        return "Sandy Clay Loam"
    elif sand < 45 and silt < 45 and 20 <= clay <= 35:
        return "Sandy Clay Loam"
    elif  sand < 30 and 40 <= silt <= 55 and 20 <= clay <= 30:
        return "Silt Loam"
    elif 30 <= sand <= 35 and silt > 50 and clay < 20:
        return "Silt Loam"
    elif 20 <= sand < 30 and silt > 47 and clay < 25:
        return "Silt"
    elif sand < 20 and silt < 20 and clay > 60:
        return "Clay"
    elif clay > 40 and sand > 20 and silt < 20:
        return "Sandy Clay"
    else:
        return "Unknown"  # Final check; if still unclassified, keep as Unknown

# First pass: Apply primary classification
with arcpy.da.UpdateCursor(shapefile, [clay_field, silt_field, sand_field, texture_field]) as cursor:
    for row in cursor:
        clay, silt, sand = row[:3]
        texture = primary_classify_texture(clay, silt, sand)
        row[3] = texture
        cursor.updateRow(row)

# Second pass: Apply secondary classification for "Unknown" values
with arcpy.da.UpdateCursor(shapefile, [clay_field, silt_field, sand_field, texture_field]) as cursor:
    for row in cursor:
        if row[3] == "Unknown":  # Only reprocess "Unknown" values
            clay, silt, sand = row[:3]
            texture = secondary_classify_texture(clay, silt, sand)
            row[3] = texture
            cursor.updateRow(row)

# Stop the edit session
edit.stopOperation()
edit.stopEditing(True)

print("Texture field populated successfully with additional classification steps.")




Texture field populated successfully with additional classification steps.
